In [1]:
import requests
import json
import prettytable
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#create dataframe from excel file May 2018
file_nat = "data_files/all_data_M_2018.xlsx"
bls_all_may2018= pd.read_excel(file_nat)
bls_all_may2018.head()

,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,99,U.S.,1,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,total,144733270,0.1,NaN,NaN,NaN,24.98,51960,0.1,9.95,12.37,18.58,30.06,47.31,20690,25740,38640,62510,98410,NaN,NaN
1,99,U.S.,1,000000,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,7616650,0.2,NaN,NaN,NaN,58.44,121560,0.1,23.68,34.08,50.11,72.93,#,49260,70880,104240,151700,#,NaN,NaN
2,99,U.S.,1,000000,Cross-industry,cross-industry,1235,11-1000,Top Executives,minor,2535640,0.3,NaN,NaN,NaN,61.66,128240,0.2,20.86,31.73,49.73,78.81,#,43400,66000,103450,163930,#,NaN,NaN
3,99,U.S.,1,000000,Cross-industry,cross-industry,1235,11-1010,Chief Executives,broad,195530,0.7,NaN,NaN,NaN,96.22,200140,0.5,32.86,55.75,91.15,#,#,68360,115960,189600,#,#,NaN,NaN
4,99,U.S.,1,000000,Cross-industry,cross-industry,1235,11-1011,Chief Executives,detailed,195530,0.7,NaN,NaN,NaN,96.22,200140,0.5,32.86,55.75,91.15,#,#,68360,115960,189600,#,#,NaN,NaN


In [5]:
bls_all_may2018.tail(5)

,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
403890,4800004,Hill Country Region of Texas nonmetropolitan area,6,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",detailed,40,32.8,0.193,1.04,NaN,13.41,27900,7.3,10.16,11.15,13.27,14.99,17.09,21130,23190,27600,31170,35550,NaN,NaN
403891,5500001,Northwestern Wisconsin nonmetropolitan area,6,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",detailed,170,25.8,3.473,18.61,NaN,11.74,24420,4.9,8.73,10.06,11.93,13.41,14.48,18160,20920,24820,27880,30130,NaN,NaN
403892,5500002,Northeastern Wisconsin nonmetropolitan area,6,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",detailed,170,41.4,0.863,4.63,NaN,12.91,26850,6.6,10.11,10.75,11.82,14.23,18.17,21020,22360,24590,29590,37790,NaN,NaN
403893,5500003,South Central Wisconsin nonmetropolitan area,6,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",detailed,130,29.2,0.632,3.39,NaN,12.35,25690,6.7,9.03,10.23,12.84,14.41,15.45,18790,21280,26710,29970,32130,NaN,NaN
403894,5500004,Western Wisconsin nonmetropolitan area,6,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",detailed,**,**,**,**,NaN,11.93,24810,6.3,8.8,9.99,10.95,11.91,18.29,18290,20770,22770,24760,38040,NaN,NaN


In [6]:
#Count number of distinct elements in each column in this df.
bls_all_may2018.nunique()

area              584
area_title        584
area_type           5
naics             420
naics_title       398
i_group             7
own_code           10
occ_code         1373
occ_title        1103
o_group             5
tot_emp         11987
emp_prse          501
jobs_1000       26869
loc_quotient     2093
pct_total        2819
h_mean           9719
a_mean          17193
mean_prse         300
h_pct10          5599
h_pct25          6818
h_median         7850
h_pct75          8729
h_pct90          9053
a_pct10          9900
a_pct25         12192
a_median        14601
a_pct75         17070
a_pct90         18460
annual              1
hourly              1
dtype: int64

In [7]:
#to see all columns in the df
list_all_may2018 = bls_all_may2018.columns.to_list()
list_all_may2018

['area',
 'area_title',
 'area_type',
 'naics',
 'naics_title',
 'i_group',
 'own_code',
 'occ_code',
 'occ_title',
 'o_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'pct_total',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'annual',
 'hourly']

In [8]:
# converting the table heading to lowercase
list_all_may2018_lowcase = [col.lower() for col in list_all_may2018]
list_all_may2018_lowcase

['area',
 'area_title',
 'area_type',
 'naics',
 'naics_title',
 'i_group',
 'own_code',
 'occ_code',
 'occ_title',
 'o_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'pct_total',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'annual',
 'hourly']

In [9]:
bls_all_may2018.columns = list_all_may2018_lowcase

In [18]:
# drop unwanted columns
bls_all_may2018_clean = bls_all_may2018.drop(['area_type',
 'naics',
 'naics_title',
 'i_group',
'own_code',
 'jobs_1000',
 'loc_quotient',
'pct_total',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'annual',
 'hourly'] ,axis=1)
bls_all_may2018_clean.head()

,area,area_title,occ_code,occ_title,o_group,tot_emp,emp_prse,h_mean,a_mean,mean_prse
0,99,U.S.,00-0000,All Occupations,total,144733270,0.1,24.98,51960,0.1
1,99,U.S.,11-0000,Management Occupations,major,7616650,0.2,58.44,121560,0.1
2,99,U.S.,11-1000,Top Executives,minor,2535640,0.3,61.66,128240,0.2
3,99,U.S.,11-1010,Chief Executives,broad,195530,0.7,96.22,200140,0.5
4,99,U.S.,11-1011,Chief Executives,detailed,195530,0.7,96.22,200140,0.5


In [19]:
bls_all_may2018_clean.isna().sum()

area          0
area_title    0
occ_code      0
occ_title     0
o_group       0
tot_emp       0
emp_prse      0
h_mean        0
a_mean        0
mean_prse     0
dtype: int64

In [20]:
# Convert columns to numeric 
cols = ['tot_emp',
 'emp_prse',
 'h_mean',
 'a_mean',
 'mean_prse',
 ]

for col in cols:
    print(col)
    bls_all_may2018_clean[col] = pd.to_numeric( bls_all_may2018_clean[col], errors='coerce')

bls_all_may2018_clean.head()

tot_emp
emp_prse
h_mean
a_mean
mean_prse


,area,area_title,occ_code,occ_title,o_group,tot_emp,emp_prse,h_mean,a_mean,mean_prse
0,99,U.S.,00-0000,All Occupations,total,144733270.0,0.1,24.98,51960.0,0.1
1,99,U.S.,11-0000,Management Occupations,major,7616650.0,0.2,58.44,121560.0,0.1
2,99,U.S.,11-1000,Top Executives,minor,2535640.0,0.3,61.66,128240.0,0.2
3,99,U.S.,11-1010,Chief Executives,broad,195530.0,0.7,96.22,200140.0,0.5
4,99,U.S.,11-1011,Chief Executives,detailed,195530.0,0.7,96.22,200140.0,0.5


In [21]:
bls_all_may2018_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403895 entries, 0 to 403894
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   area        403895 non-null  int64  
 1   area_title  403895 non-null  object 
 2   occ_code    403895 non-null  object 
 3   occ_title   403895 non-null  object 
 4   o_group     403895 non-null  object 
 5   tot_emp     374496 non-null  float64
 6   emp_prse    374496 non-null  float64
 7   h_mean      382544 non-null  float64
 8   a_mean      396782 non-null  float64
 9   mean_prse   397000 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 30.8+ MB


In [22]:
# remove NAN in this dataframe 
bls_all_may2018_clean.dropna()

,area,area_title,occ_code,occ_title,o_group,tot_emp,emp_prse,h_mean,a_mean,mean_prse
0,99,U.S.,00-0000,All Occupations,total,144733270.0,0.1,24.98,51960.0,0.1
1,99,U.S.,11-0000,Management Occupations,major,7616650.0,0.2,58.44,121560.0,0.1
2,99,U.S.,11-1000,Top Executives,minor,2535640.0,0.3,61.66,128240.0,0.2
3,99,U.S.,11-1010,Chief Executives,broad,195530.0,0.7,96.22,200140.0,0.5
4,99,U.S.,11-1011,Chief Executives,detailed,195530.0,0.7,96.22,200140.0,0.5
...,...,...,...,...,...,...,...,...,...,...
403889,4800002,North Texas Region of Texas nonmetropolitan area,53-7199,"Material Moving Workers, All Other",detailed,190.0,20.1,19.39,40320.0,11.8
403890,4800004,Hill Country Region of Texas nonmetropolitan area,53-7199,"Material Moving Workers, All Other",detailed,40.0,32.8,13.41,27900.0,7.3
403891,5500001,Northwestern Wisconsin nonmetropolitan area,53-7199,"Material Moving Workers, All Other",detailed,170.0,25.8,11.74,24420.0,4.9
403892,5500002,Northeastern Wisconsin nonmetropolitan area,53-7199,"Material Moving Workers, All Other",detailed,170.0,41.4,12.91,26850.0,6.6


In [23]:
bls_all_may2018_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403895 entries, 0 to 403894
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   area        403895 non-null  int64  
 1   area_title  403895 non-null  object 
 2   occ_code    403895 non-null  object 
 3   occ_title   403895 non-null  object 
 4   o_group     403895 non-null  object 
 5   tot_emp     374496 non-null  float64
 6   emp_prse    374496 non-null  float64
 7   h_mean      382544 non-null  float64
 8   a_mean      396782 non-null  float64
 9   mean_prse   397000 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 30.8+ MB


In [24]:
bls_all_may2018_clean.head(5)

,area,area_title,occ_code,occ_title,o_group,tot_emp,emp_prse,h_mean,a_mean,mean_prse
0,99,U.S.,00-0000,All Occupations,total,144733270.0,0.1,24.98,51960.0,0.1
1,99,U.S.,11-0000,Management Occupations,major,7616650.0,0.2,58.44,121560.0,0.1
2,99,U.S.,11-1000,Top Executives,minor,2535640.0,0.3,61.66,128240.0,0.2
3,99,U.S.,11-1010,Chief Executives,broad,195530.0,0.7,96.22,200140.0,0.5
4,99,U.S.,11-1011,Chief Executives,detailed,195530.0,0.7,96.22,200140.0,0.5


In [25]:
#create CSV files to be used to load database as needed
bls_all_may2018_clean.to_csv('data_files/bls_all_may2018_clean.csv')
